# select significant electrodes for cue and read task

In [1]:
import numpy as np
from scipy import stats
import os
bands_list = ['theta', 'alpha', 'beta1', 'beta2', 'low gamma', 'high gamma']
freq_list=[400]
HS_list=[68, 69, 75, 79, 82, 83, 84, 85, 86]

In [2]:
def load_band_data(data_path, HS, band):
    keys = ['功课', '力果', '宫客', '作业', 'gōng kè', '树叶', '对十', '数页', '绿草', 'shù yè']

    # 加载每个 band 的 block 数据
    file_path = os.path.join(data_path, f"HS{HS}block_{band}.npy")
    block_data = np.load(file_path, allow_pickle=True).item()

    for key in keys:
        if key in block_data:
            data = block_data[key]  # data 是一个长度为 256 的列表，每个元素 shape 为 (time2200, trials)
            reshaped_data = np.array([elec_data.T for elec_data in data])  # (electrodes, trials, time)
            reshaped_data = reshaped_data.transpose(1, 0, 2)  # (trials, electrodes, time)
    
    return reshaped_data
                
import numpy as np
from scipy import stats

def calculate_p_values(ecogReading_task):
    """
    计算每个电极在每个时间点的 p 值。
    参数:
    - ecogReading_task: numpy.ndarray, shape (n_trials, n_electrodes, n_times)
    返回:
    - p_values: numpy.ndarray, shape (n_electrodes, n_times)
    """
    # 获取形状信息
    n_trials, n_electrodes, n_times = ecogReading_task.shape

    # 定义基线时间窗（例如 -200ms 到 0ms 对应的时间点范围）
    start = int(1 * 400 - 0.2 * 400)  # 转换到采样点
    end = start + 80
    p_values = np.zeros((n_electrodes, n_times))

    for electrode_idx in range(n_electrodes):
        electrode_data = ecogReading_task[:, electrode_idx, :]
        baseline = electrode_data[:, start:end].reshape(-1)
        for time_idx in range(400, n_times):
            current_data = electrode_data[:, time_idx]
            baseline_valid = baseline[~np.isnan(baseline)]
            current_data_valid = current_data[~np.isnan(current_data)]
            if len(current_data_valid) > 0 and len(baseline_valid) > 0:
                p_values[electrode_idx, time_idx] = stats.ttest_ind(
                    baseline_valid, current_data_valid, alternative='less'
                ).pvalue
            else:
                p_values[electrode_idx, time_idx] = np.nan
                
    return p_values


In [ ]:
base_path = '/public/DATA/overt_reading/Blocked_data/'  # 数据基础路径
save_base_path = '/public/DATA/overt_reading/t-test_results/'
for HS in HS_list:
    os.makedirs(os.path.join(save_base_path, f'HS{HS}'), exist_ok=True)
    band_p_values = {}
    for band in bands_list:
        band_ecog_data = load_band_data(base_path, HS, band)
        p_value = calculate_p_values(band_ecog_data)
        band_p_values[band] = p_value
    np.save(os.path.join(save_base_path, f'HS{HS}/allband_p_values.npy'), band_p_values)